In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from decimal import Decimal, ROUND_HALF_UP

from .utils import baseline_utils

GRANULARITY = 100
BOUNDARIES = (-1000, 1000)
DRAWS = False

In [ ]:
baseline = pd.read_csv(r'..\..\data\baseline-june.csv')

if not DRAWS:
    baseline['Total Points'] -= .5*baseline['Total Draws']
    baseline['Total Games'] -= baseline['Total Draws']

df = pd.DataFrame(columns=baseline.columns)

for gs in baseline['Game Speed'].unique():
    df.loc[len(df)] = columns_group_tail(baseline, gs, BOUNDARIES[0], upper_or_lower='lower')
    for num in range(BOUNDARIES[0], BOUNDARIES[1]+1, GRANULARITY):
        df.loc[len(df)] = columns_group(baseline, gs, GRANULARITY, num)
    df.loc[len(df)] = columns_group_tail(baseline, gs, BOUNDARIES[1], upper_or_lower='upper')

df['Game Speed'] = pd.Categorical(df['Game Speed'], ["Bullet", "Blitz", "Rapid", "Classical"])
df.sort_values(by=['Game Speed', 'Rating Difference'], inplace=True)
        
interval = con_interval(df['Total Points'], df['Total Games'])
df['White Win Probability'] = interval[0]
df['Lower White Win Probability'] = interval[1]
df['Upper White Win Probability'] = interval[2]

interval = con_interval(df['Total Draws'], df['Total Games'])
df['Draw Probability'] = interval[0]
df['Lower Draw Probability'] = interval[1]
df['Upper Draw Probability'] = interval[2]

float_columns = [column for column in df.columns if column not in ['Game Speed', 'Rating Difference', 'Total Games', 'Total Draws']]
int_columns = ['Total Games', 'Total Draws']
df[float_columns] = df[float_columns].astype(float)
df[int_columns] = df[int_columns].astype(int)

clip_columns = ['Lower White Win Probability', 'Upper White Win Probability', 'Lower Draw Probability', 'Upper Draw Probability']
df[clip_columns] = df[clip_columns].clip(0, 1)

In [ ]:
for attribute in df.columns[5:]:
    file_name = "".join([word[0].lower() for word in attribute.split(" ")]) + "_" + str(BOUNDARIES[1]) + "_" + str(GRANULARITY)
    create_heatmap(create_pivot(df, 'Rating Difference', 'Game Speed', attribute, BOUNDARIES), 'Rating Difference', 'Game Speed', attribute, GRANULARITY)
    plt.savefig(r"..\..\results/" + file_name, facecolor='w', bbox_inches="tight")
    plt.show()